In [1]:
from hatchet import *
import pandas as pd
import numpy as np
from state import State
import matplotlib.pyplot as plt
import json
import utils

In [2]:
%matplotlib inline

In [4]:
pd.options.display.max_rows = 75
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (16, 12)

In [10]:
datasets = ['calc-pi', 'calc-pi-half']
dir_name = "/home/vidi/Work/llnl/CallFlow/.callflow/"
# dir_name = "/Users/jarus/ucd/Research/Visualisation/projects/CallFlow/.callflow/"

In [7]:
def replace_str_with_Node(df, graph):
        mapper = {}
        def dfs_recurse(root):
            for node in root.children: 
                mapper[node.callpath[-1]] = Node(node.nid, node.callpath, None)
                dfs_recurse(node)
        for root in graph.roots:
            mapper[root.callpath[-1]] = Node(root.nid, root.callpath, None)
            dfs_recurse(root)
        df['node'] = df['node'].apply(lambda node: mapper[node] if node in mapper else '')
        return df

In [12]:
def read_gf(name):
        state = State()
        path = dir_name + name
        df_filepath = path + '/filter_df.csv'
        entire_df_filepath = path + '/entire_df.csv'
        graph_filepath = path + '/filter_graph.json'
        entire_graph_filepath = path + '/entire_graph.json'   

        with open(graph_filepath, 'r') as graphFile:
            data = json.load(graphFile)

        state.gf = GraphFrame()
        state.gf.from_literal(data)

        with open(entire_graph_filepath, 'r') as entire_graphFile:
            entire_data = json.load(entire_graphFile)
            
        state.entire_gf = GraphFrame()
        state.entire_gf.from_literal(entire_data)

        state.df = pd.read_csv(df_filepath)
        state.entire_df = pd.read_csv(entire_df_filepath)

        state.graph = state.gf.graph
        state.entire_graph = state.entire_gf.graph

#         state.map = state.node_hash_mapper()

        # Print the module group by information. 
        # print(state.df.groupby(['module']).agg(['mean','count']))

        # replace df['node'] from str to the Node object.
        state.df = replace_str_with_Node(state.df, state.graph)
        state.entire_df = replace_str_with_Node(state.entire_df, state.entire_graph)

        # add path to the dataframes. 
        # state.df['path'] = state.df['node'].apply(lambda node: node.callpath)
        # state.entire_df['path'] = state.entire_df['node'].apply(lambda node: node.callpath if node else [])

        return state

In [14]:
states = {}
for dataset in datasets:
    states[dataset] = read_gf(dataset)
print(states[datasets[0]].g)

None
